In [ ]:
from pyspark import SparkConf
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import explode, split, col, sum, lit
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
        .appName("app_name") \
        .getOrCreate()

In [ ]:
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "lakefs")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "lakefs")
#spark._jsc.hadoopConfiguration().set("fs.s3a.session.token", "")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
#spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "endpoint")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
#df = spark.read.csv('s3a://lakefs/localM2-randread.csv', header=True, schema=custom_schema)

In [ ]:
custom_schema = StructType([
                StructField("bs", StringType(), True),
                StructField("iodepth", StringType(), True),
                StructField("bw_mean", DoubleType(), True),
                StructField("bw_min", DoubleType(), True),
                StructField("bw_max", DoubleType(), True),
                StructField("bw_dev", DoubleType(), True),
                StructField("iops_mean", IntegerType(), True),
                StructField("iops_min", IntegerType(), True),
                StructField("iops_max", IntegerType(), True),
                StructField("iops_stddev", DoubleType(), True)])

In [ ]:
df = spark.read.csv('s3a://lakefs/moderatelyBig.csv', header=True, schema=custom_schema)

In [ ]:
df.show()

In [ ]:
%%bash
s3cmd -s --access_key=lakefs --secret_key=lakefs --host=endpoint:443 --no-check-certificate la